In [2]:
# Imports for JupyterLite
try:
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass

!pip install gspread
import gspread

#!pip install --upgrade ipywidgets
import ipywidgets as widgets
# Utils widgets
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Box, HBox, VBox

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

import numpy as np
import re

In [3]:
#debug functions

def debug_log(text):
    if debug == True:
        print(text)
        return

In [4]:
#turns printing to debug log on (True) or off (False)
debug = True

In [15]:
def VBox(*pargs, **kwargs):
    """Displays multiple widgets vertically using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.flex_flow = 'column'
    box.layout.align_items = 'stretch'
    return box

def HBox(*pargs, **kwargs):
    """Displays multiple widgets horizontally using the flexible box model."""
    box = Box(*pargs, **kwargs)
    box.layout.display = 'flex'
    box.layout.align_items = 'stretch'
    return box

def add_widget_to_Box(widget, Box):
    box_children_list = list(Box.children)
    box_children_list.append(widget)
    Box.children = tuple(box_children_list)
    return

def remove_widget_from_Box(widget, Box):
    box_children_list = list(Box.children)
    box_children_list.remove(widget)
    Box.children = tuple(box_children_list)
    return

def remove_last_widget_from_Box(Box):
    box_children_list = list(Box.children)
    box_children_list.pop()
    Box.children = tuple(box_children_list)
    return

In [61]:
class filter_ui:
    #google sheet variables
    google_sa = None
    sheet = None
    worksheet = None
    
    #change these values to change which sheet and worksheet you want to open
    sheet_name = "Venus Blood Savior"
    worksheet_name = "Recruitable Units"
    other_worksheet_names = ["Talismans","Equipment"]
    other_worksheet_dfs = None
    
    """
    in the form:
    'skill_name': [
        "attack_bonus_percentage",
        "defense_bonus_percentage",
        "speed_bonus_percentage",
        "wisdom_bonus_percentage",
        "damage_bonus_percentage",
        "damage_reduction_percentage"
        ]
    make sure all 6 slots are at least filled with 0 for no value or function will break
    """
    skill_dict = {'skill1': [10, 20, 30, 40, 50, 60],
                  'skill2': [15, 25, 35, 45, 55, 65]
                 }
    
    #dataframe variables
    unit_df = None
    filter_df = None
    
    #widget variables
    sheet_name_label = None
#     add_category_button = None
    
    #layout variables
#     header_section_VBox = None # for expanding header in the future possibly
    filter_section_VBox = None
    wrapper_VBox = None
    
    #layout helper variables
    filter_rows = None
    
    #category option variables
#     valid_categories = []
    forbidden_categories = ['Name']
#     selected_categories = None
#     selected_tags= []

    filter_dict = None
    
    def __init__(self):
        
        """
        begin code for importing google sheets
        
        follow these tutorials for to set up google sheets yourself:
        https://ploomber.io/blog/gsheets/
        https://towardsdatascience.com/from-google-sheet-to-your-jupyter-notebook-ccdbf28fbf1b

        share any sheets you want to use with the service email you made from the tutorial (make sure to give edit permissions)
            should look like: list-381@list-380300.iam.gserviceaccount.com

        sheetname: change this to the name of the spreadsheet you want to open
        worksheetname: change this to the name of the spreadsheet tab/worksheet you want to open
        """

        #use the service account file to authenticate, download this file to your pc in the same folder as this jupyter notebook
        self.google_sa = gspread.service_account(filename="list-380300-d89958bc04be.json")
        
        self.sheet = self.google_sa.open(self.sheet_name)
        self.worksheet = self.sheet.worksheet(self.worksheet_name)
        
        self.other_worksheet_dfs = {}
        for name in self.other_worksheet_names:
            df = pd.DataFrame(self.sheet.worksheet(name).get_all_records())
            self.other_worksheet_dfs.update({name: df})
#         debug_log(self.other_worksheet_dfs)

        self.unit_df = pd.DataFrame(self.worksheet.get_all_records())
        
        #end code for importing google sheets
        
        self.sheet_name_label = widgets.Label(
            value="Sheet Name: " + self.sheet_name,
            style=dict(
                font_size='64',
                font_weight='bold')
        )

#         self.add_category_button = widgets.Button(
#                 description='Add category',
#                 disabled=False,
#                 button_style='info', # 'success', 'info', 'warning', 'danger' or ''
#                 tooltip='Click to add category',
#                 icon='plus', # (FontAwesome names without the `fa-` prefix)
#                 layout=Layout(height='auto', width='auto')
#         )
        
#         self.selected_categories = []
        self.filter_dict = {}
        
        self.filter_rows = []
        new_filter_row = filter_row(self, 0, self.create_valid_category_list())
        self.filter_rows.append(new_filter_row)
        self.filter_section_VBox = VBox(children=[new_filter_row.wrapper_VBox])
#         self.filter_rows.append(filter_row(self, 0, self.create_valid_category_list()))
#         self.filter_section_VBox = VBox(children=[self.filter_rows[0].wrapper_VBox])
        
        self.wrapper_VBox = VBox(children=[self.sheet_name_label, self.filter_section_VBox])
#         self.wrapper_VBox = VBox(children=[self.sheet_name_label, self.add_category_button, self.filter_section_VBox])
        
        #add event handlers
#         self.add_category_button.on_click(self.add_filter_row)
        
    #event handler functions
    def add_filter_row(self,button):
        if self.is_categories_all_selected():
            new_filter_row = filter_row(self, len(self.filter_rows), self.create_valid_category_list())
            self.filter_rows.append(new_filter_row)
            add_widget_to_Box(new_filter_row.wrapper_VBox, self.filter_section_VBox)
#             self.filter_rows.append(filter_row(self, len(self.filter_rows), self.create_valid_category_list()))
#             add_widget_to_Box(self.filter_rows[(len(self.filter_rows)-1)].wrapper_VBox, self.filter_section_VBox)
        
    #check if categories in filter rows have all been selected
    def is_categories_all_selected(self):
        if len(self.create_valid_category_list()) == 0:
            return False
        for filter_row in self.filter_rows:
            if filter_row.category_combobox.disabled == False:
                return False
        return True
        
    #data operation functions
    def get_unique_characters(self, s):
        # Convert each array to a list of strings and join them
        joined_str = ''.join([','.join(x) for x in s.tolist()])
        # Split the resulting string into individual characters and get the unique ones
        return list(set(joined_str))

    def get_unique_strings(self, column_name):
        column=self.unit_df[column_name].str.split(',')

        # flatten the arrays into a single list of strings
        flat_column = column.explode().tolist()

        # preprocess the strings to remove numbers in square brackets
        flat_column = [re.sub(r'\[\d+\]', '', s) for s in flat_column]

        # extract unique values and convert to list
        unique_strings = pd.unique(flat_column).tolist()

        return unique_strings
    
    def get_skills_by_category(self, worksheet_name, category, value):
        """
        Returns a list of possible skills that a letter growth rate can use.
        Args:
            worksheet_name (str): The name of the worksheet to search for the value in, ex. 'Growth Rate'
            value: The value to search for, ex. 'E' for growth rate
        Returns:
            skills (list): A list of possible skills that the growth rate can use.
        """
        # Filter the DataFrame to only include rows with the specified growth rate
        df = self.other_worksheet_dfs[worksheet_name]
        
        skills_df = df[df[category] == value]

        # Extract the Skill column from the filtered DataFrame
        skills = skills_df['Skills'].tolist()

        return skills
    
    def get_possible_skills(self, **kwargs):
        """
        Returns an array of all possible skills looking in growth rate talismans and equipment slots.
        Args:
            kwargs: keyword argument where the dictionary key gives the value, pass in args in the form: "slot = '片手'"
        Returns:
            skills (set): An set of skills that are possible.
        """
        
        talisman_skills = self.get_skills_by_category(kwargs['wkst_1'], kwargs['cat_1'], kwargs['growth_rate'])
        t_skills_set = set(talisman_skills)
        
        slot_skills = self.get_skills_by_category(kwargs['wkst_2'], kwargs['cat_2'],kwargs['slot'])
        s_skills_set = set(slot_skills)
        
        skills_set = t_skills_set.union(s_skills_set)
        
        return skills_set
    
#     def get_data_from_filter_rows(self):
# #         debug_log(self.filter_rows)
#         self.filter_dict = {}
#         for filter_row in self.filter_rows:
#             self.filter_dict.update({filter_row.category_combobox.value: filter_row.current_tags})
# #         debug_log('get data end')
# #         debug_log(self.filter_dict)

#     def filter_df_by_selected_tags(self, df):
#         self.filter_df=df
#         self.get_data_from_filter_rows()
#         #debug_log(self.filter_rows)
#         for category in self.filter_dict:
#             for tag in self.filter_dict[category]:
#                 self.filter_df = self.filter_df[self.filter_df[category].str.contains(tag)]
# #         debug_log('filter attempt end')

    def filter_df_by_selected_tags(self, df):
        self.filter_df=df
        #debug_log(self.filter_rows)
        for tag in self.filter_rows[0].current_tags:
            category = self.filter_rows[0].current_tags[tag]
            self.filter_df = self.filter_df[self.filter_df[category].str.contains(tag)]
#         debug_log('filter attempt end')

    """
    returns an array of effective stats:
    effective stats = [
        effective_attack,
        effective_defense,
        effective_speed,
        effective_wisdom,
        effective_damage_bonus,
        effective_damage_reduction]
    """
    def calculate_effective_stats(self, unit):
        unit_row =  ui.unit_df.loc[(ui.unit_df["Name"]==unit)]
#         display(unit_row)
        print(unit_row.at[1,"Attack"])
        unit_stats = {
            "Attack": unit_row.at[1,"Attack"],
            "Defense": unit_row.at[1,"Defense"],
            "Speed": unit_row.at[1,"Speed"],
            "Wisdom": unit_row.at[1,"Wisdom"],
            "Damage Bonus": 0,
            "Damage Reduction": 0
        }
        unit_skill_bonuses = self.get_skills_from_unit(unit)
        effective_stats = {
            "Attack": unit_row.at[1,"Attack"],
            "Defense": unit_row.at[1,"Defense"],
            "Speed": unit_row.at[1,"Speed"],
            "Wisdom": unit_row.at[1,"Wisdom"],
            "Damage Bonus": 0,
            "Damage Reduction": 0
        }
        for skill in unit_skill_bonuses:
            bonuses = unit_skill_bonuses[skill]
            
            effective_stats["Attack"] = (effective_stats["Attack"] * bonuses[0])
            effective_stats["Defense"] = (effective_stats["Defense"] * bonuses[1])
            effective_stats["Speed"] = (effective_stats["Speed"] * bonuses[2])
            effective_stats["Wisdom"] = (effective_stats["Wisdom"] * bonuses[3])
            effective_stats["Damage Bonus"] = (effective_stats["Damage Bonus"] * bonuses[4])
            effective_stats["Damage Reduction"] = (effective_stats["Damage Reduction"] * bonuses[5])
        return effective_stats

    def get_skills_from_unit(self, unit):
        unit_skills = self.skill_dict
        return unit_skills
    
#     #valid category selection option filtering functions
#     def is_valid_category(self, category):
#         if category not in self.forbidden_categories and category not in self.selected_categories:
#     #         debug_log('valid')
#             return True
#     #     debug_log('not valid')
#         return False
    
    #valid category selection option filtering functions
    def is_valid_category(self, category):
        if category not in self.forbidden_categories:
    #         debug_log('valid')
            return True
    #     debug_log('not valid')
        return False
    
    def create_valid_category_list(self):
        non_numeric_columns=self.unit_df.select_dtypes(exclude=[np.number]).columns.tolist()
        valid_categories = []
        for column in non_numeric_columns:
            if self.is_valid_category(column):
                valid_categories.append(column)
        return valid_categories
    
    #filter selectable tags by category function
    def get_tags_of_category(self, category):
    #     debug_log('getting tags of category')
        category_tags = self.get_unique_strings(category)
        return category_tags
        
    #display functions
    def display(self):
        display(self.wrapper_VBox)
        
class filter_row:
    parent_ui = None
    wrapper_VBox = None
    index = 0

    category_and_tag_selection_HBox = None
    category_combobox = None
    tag_dropdown = None

    current_tags_HBox = None
    tags_label = None
    current_tags = None

    category_options = None
    tag_options = None

    def __init__(self, parent, idx, cat_options):
        self.parent_ui = parent
        
        self.index = idx
        
        self.category_options = cat_options
        #             self.tag_options = tag_options
        self.current_tags = {} #current_tag list MUST be declared here, if declared in class variables all instances will point to the same list object

        self.category_combobox = widgets.Combobox(
            # value=poptions[0],
            placeholder='Choose category',
            options=self.category_options,
            description='Look in category: ',
            ensure_option=True,
            disabled=False,
            style=dict(description_width='initial')
        )

        self.tags_label = widgets.Label(
            value="Category tags: ",
            style=dict(font_weight='bold')
        )

        self.current_tags_HBox = HBox(children=[self.tags_label])

        self.category_and_tag_selection_HBox = HBox(children=[self.category_combobox])
        
        self.wrapper_VBox = VBox(children=[self.category_and_tag_selection_HBox,self.current_tags_HBox])

        #add event handlers
        self.category_combobox.observe(self.on_category_selected, names = 'value')

    #event handler functions
#     def on_category_selected(self,change):
#         self.add_tag_dropdown()
#         self.parent_ui.selected_categories.append(change.new)
#         change.owner.disabled = True
#         #             debug_log(change.owner)
        
    def on_category_selected(self,change):
        self.add_tag_dropdown()
        
        #             debug_log(change.owner)

    #tag dropdown creation functions (uses parent_ui)
    def add_tag_dropdown(self):
        if len(self.category_and_tag_selection_HBox.children) == 2:
            remove_last_widget_from_Box(self.category_and_tag_selection_HBox)
        self.tag_options = self.parent_ui.get_tags_of_category(self.category_combobox.value) #get tags of category from parent_ui
        self.tag_dropdown = widgets.Dropdown(
            options=self.tag_options,
            value=self.tag_options[0],
            description='for tag:',
            disabled=False,
            style=dict(description_width='initial')
        )

        add_widget_to_Box(self.tag_dropdown,self.category_and_tag_selection_HBox)
        
        #add event handlers
        self.tag_dropdown.observe(self.add_selected_tag, names = 'value')
        
    #tag selection and creation functions
    def add_selected_tag(self, change):
#         debug_log(change.new)
        if change.new not in self.current_tags:
            add_widget_to_Box(self.create_tag_button(change.new), self.current_tags_HBox)
            self.current_tags[change.new] = self.category_combobox.value
            debug_log(self.current_tags)        
    
    #create and observe tag button (uses parent_ui)
    def create_tag_button(self, tag_name):
        tag_button = widgets.Button(
            description=tag_name,
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click to remove',
            icon='close', # (FontAwesome names without the `fa-` prefix)
            layout=Layout(height='auto', width='auto')
        )
#         self.current_tags.append(tag_name)
#         self.parent_ui.selected_tags.append(tag_name) #adds to masterlist of selected tags in parent_ui
#         debug_log(self.current_tags)
        tag_button.on_click(self.remove_tag)
        return tag_button
    
    #event handler for removing tag button (uses parent_ui)
    def remove_tag(self, button):
        remove_widget_from_Box(button, self.current_tags_HBox)
        del self.current_tags[button.description]
#         self.current_tags.remove(button.description)
#         self.parent_ui.selected_tags.remove(button.description) #removes from masterlist of selected tags in parent_ui
        button.close()

In [62]:
ui = filter_ui()
ui.display()

Box(children=(Label(value='Sheet Name: Venus Blood Savior'), Box(children=(Box(children=(Box(children=(Combobo…

{'土': 'Element'}
{'土': 'Element', 'ガーダー': 'Type'}
{'土': 'Element', 'ガーダー': 'Type', '女': 'Race'}


In [65]:
ui.filter_df_by_selected_tags(ui.unit_df)
# ui.filter_df[["Name", ui.selected_categories]]
display(ui.filter_df)

,Name,Element,Type,HP,Attack,Defense,Speed,Wisdom,Race,Race Advantage,Equipment Slots,Growth Rate,Cost,Skills,Leader Skills,Assist Skill,Governance Skill,Stance Skills,Recruitment Talismans,Recruitment Category
46,ホルスタウロス,土,ガーダー,100,30,37,10,12,"女,魔,獣,陸,騎","樹,器","斧槍,盾",S,5,"反撃倍加[3],貫通無効,獣族活性[8],堅守体躯[20],防御布陣[12],専守防衛,ブロック[20],特攻防御[25]","パリング[30],前進防御",堅守体躯[1],農業[20],"防備,混沌","獣,闘,信,巨",亜人
167,スイカオーランタン,土,ガーダー,110,32,53,9,3,"女,樹,騎","獣,陸,蟲","片手,盾",A,10,"対術吸収[35],防御布陣[20],反撃倍加[6],必殺増加[35],リカバリ[35],専守防衛,堅守体躯[25],加速進化[8]","守勢転化[30],異常耐性",バリアー[1],農業[16],"防備,秩序","力,陸,闘,命",精霊
615,レディフォートレス,土,ガーダー,105,19,46,6,9,"人,女,騎","騎,死,竜","斧槍,盾",S,6,"防御布陣[16],前進防御,城壁構築[75],堅守体躯[15],貫通無効,ブロック[35],専守防衛,砲撃結界[25]","砲撃障壁[75],扇形無効",必殺耐性[10],軍事[3],"防備,自由","技,陸,巨,歴",人間
693,熾天使ウリエル,"光,土",ガーダー,135,90,135,50,15,"女,神,飛,器,騎,超","魔,神,器,海","斧槍,盾",D,26,"奇襲警戒[30],防御布陣[60],前進防御,次元障壁[50],専守防衛,堅守体躯[45],特攻防御[60],十字無効","全域無効,リカバリ[50]",必殺耐性[10],軍事[3],"防備,秩序","巨,英,王,使,神,刹,創",神族
